# 네이버 지도 크롤링

검색어에 따른 장소에 대한 정보를 크롤링하는 코드입니다.   
장소의 이름, 전화번호, 주소, 평점, 리뷰정보에 대해 크롤링합니다.   
장소에 대한 간단한 정보들을 담은 data, 리뷰에 대한 정보들을 담은 data, 블로그 주소에 대한 data를 가져옵니다.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import datetime

# 특정 div scroll down function
def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    cont = driver.find_element_by_id('_pcmap_list_scroll_container')
    
    while True:
        driver.execute_script("arguments[0].scrollBy(0, 10000)", cont)
        time.sleep(1)
        if datetime.datetime.now() > end:
            break


# 네이버 지도는 접속 시 자동으로 아래 url 뒤에 현재 위치 정보가 입력됨
driver_path = './driver/chromedriver'
URL = 'https://map.naver.com/v5/'
search_keyword = '카페'

driver = webdriver.Chrome(executable_path=driver_path)
driver.get(URL)

time.sleep(5)

# keyword 검색
search = driver.find_element_by_css_selector('div.input_box>input.input_search')
search.send_keys(search_keyword)
search.send_keys(Keys.ENTER)

time.sleep(3)

# frame 전환
search_frame = driver.find_element_by_id('searchIframe')
driver.switch_to.frame(search_frame)
doScrollDown(5)


# 탐색은 페이지 별로 진행합니다.
search_simple_result = []
search_review_result = []
search_blog_url_result = []
cur_page = 1
while True:
    places = driver.find_elements_by_class_name('_3nCBm')
    for pl in places:
        # 검색할 장소가 있다면 상세페이지를 엽니다.
        try:
            pl.click()
        except:
            continue
        time.sleep(3)

        # 장소 상세정보 frame으로 전환
        driver.switch_to.default_content()
        entry_frame = driver.find_element_by_id('entryIframe')
        driver.switch_to.frame(entry_frame)


        # 장소의 기본정보
        # {이름:name , 전화번호:number , 주소:addr , 평점:score , 방문자수:cnt_visitor , 블로그수:cnt_blog}
        name = driver.find_element_by_class_name('_3XamX').text
        try:
            number = driver.find_element_by_class_name('_3ZA0S').text
        except:
            number = 'none'
        addr = driver.find_element_by_class_name('_2yqUQ').text
        try:
            score = driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[1]/div/div/div[1]/div/span[1]/em').text
        except:
            score = 0
        try:
            cnt_visitor = driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[1]/div/div/div[1]/div/span[2]/a/em').text
        except:
            cnt_visitor = 0
        try:
            cnt_blog = driver.find_element_by_xpath('//*[@id="app-root"]/div/div/div[2]/div[1]/div/div/div[1]/div/span[3]/a/em').text
        except:
            cnt_blog = 0


        # 방문자 리뷰정보 [review_info]
        if cnt_visitor != 0:
            driver.find_elements_by_class_name('_2RG_o')[-2].click()
            time.sleep(3)
            # 모든 리뷰정보를 로드합니다.
            while True:
                try:
                    driver.find_element_by_link_text('더보기').click()
                    time.sleep(1)
                except:
                    break
            while True:
                try:
                    driver.find_element_by_class_name('_2vy4Q').click()
                    time.sleep(1)
                except:
                    break
            # 모든 리뷰글 정보들을 가져옵니다.
            review_text = driver.find_elements_by_class_name('_3vfQ6')
            for per in review_text:
                search_review_result.append([name, per.text])


        # 블로그 리뷰정보 [blog_url]
        if cnt_blog != 0:
            driver.find_elements_by_class_name('_3Q5_9')[-1].click()
            time.sleep(3)
            # 모든 리뷰정보를 로드합니다.
            while True:
                try:
                    driver.find_element_by_link_text('더보기').click()
                    time.sleep(1)
                except:
                    break
            # 모든 리뷰 블로그 url들을 가져옵니다.
            review_url = driver.find_elements_by_class_name('awH2_')
            for per in review_url:
                search_blog_url_result.append([name, per.find_element_by_tag_name('a').get_attribute('href')])


        # 현재 장소에 대한 정보 저장
        search_simple_result.append([name, number, addr, score, cnt_visitor, cnt_blog])
        # 정보 저장 후 검색 프레임으로 이동
        driver.switch_to.default_content()
        search_frame = driver.find_element_by_id('searchIframe')
        driver.switch_to.frame(search_frame)
    
    
    # 다음 페이지가 존재한다면 이어서 탐색합니다.
    cur_page = cur_page + 1
    print('check', cur_page)
    page_div = driver.find_element_by_class_name('_2ky45')
    try:
        page_div.find_element_by_link_text(str(cur_page)).click()
    except:
        break
    time.sleep(3)
    
    
# 검색 결과 모두 조회
# result_container = driver.find_element_by_id('_pcmap_list_scroll_container')
# result_container
# 각 url내에서 데이터 크롤링 진행
# 혹은 검색 결과창에서 그대로 크롤링

# 크롤링 한 데이터 저장

# driver 종료
driver.quit()

C:\Users\namke\AppData\Local\Temp/ipykernel_10516/326459360.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)
C:\Users\namke\AppData\Local\Temp/ipykernel_10516/326459360.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_css_selector('div.input_box>input.input_search')
C:\Users\namke\AppData\Local\Temp/ipykernel_10516/326459360.py:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_frame = driver.find_element_by_id('searchIframe')
C:\Users\namke\AppData\Local\Temp/ipykernel_10516/326459360.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  cont = driver.find_element_by_id('_pcmap_list_scroll_container')
C:\Users\namke\AppData\Local\Temp/ipykernel_10516/326459360.py:48: DeprecationWarning:

check 2


C:\Users\namke\AppData\Local\Temp/ipykernel_10516/326459360.py:136: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  page_div = driver.find_element_by_class_name('_2ky45')
C:\ProgramData\Anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:265: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


check 3


정상적으로 크롤링 되었는지 가져온 데이터들을 확인합니다.

In [4]:
print(len(search_simple_result))
print(search_simple_result)

41
[['메이25', '054-251-0525', '경북 포항시 북구 두호로6번길 17 1층 103호', '4.54', '49', '51'], ['소디스커피', '0507-1346-9667', '경북 포항시 북구 삼호로 246', '4.33', '216', '230'], ['서정빌라', 'none', '경북 포항시 북구 삼호로265번길 26', '4.64', '97', '160'], ['애니시', '0507-1438-5591', '경북 포항시 북구 학전로 142 1층 애니시', '4.5', '18', '39'], ['안녕겨울 영일대점', '0507-1370-9559', '경북 포항시 북구 삼호로 265 1층', '4.4', '74', '91'], ['투썸플레이스 포항북부점', '054-255-2631', '경북 포항시 북구 해안로 93', '4.43', '318', '35'], ['투썸플레이스 포항두호점', '0507-1411-2122', '경북 포항시 북구 두호로 26 투썸플레이스 포항두호점', '4.49', '404', '9'], ['달짝지근', '010-4668-4662', '경북 포항시 북구 삼호로265번길 23 달짝지근', '4.43', '43', '22'], ['배스킨라빈스 포항두호', '054-242-3101', '경북 포항시 북구 두호로 25 아이코아', '4.5', '854', '10'], ['빽다방 포항영일대점', '010-7271-7696', '경북 포항시 북구 삼호로 237', '4.6', '73', '9'], ['파스쿠찌 포항영일대비치점', '054-248-3303', '경북 포항시 북구 해안로 131 발렌타인호텔 1층', '4.35', '311', '14'], ['공차 포항두호점', '0507-1465-1241', '경북 포항시 북구 두호로 25', '4.45', '560', '11'], ['커피빈 포항영일대점', '054-241-3372', '경북 포항시 북구 삼호로265번길 1 NB-City 복합시설', '4.58', '327',

In [5]:
print(len(search_review_result))
print(search_review_result)

1790
[['메이25', '10명 이상 참여하면\n방문 키워드 통계가 공개됩니다!'], ['메이25', '다 맛있어요.그중 난 빠에야'], ['메이25', '너무 맛있고 기분이 좋아져요. 대접받는 느낌이 막 풍 기는 좋은 곳이었어요. 저는 가족들과 방문했지만 아 마 나중에는 친구들과 그리고 어떤때는 혼자도...\n내용 더보기'], ['메이25', '좋아요'], ['메이25', '음식은 깔끔. 직접 만든 치즈, 베이컨 등등'], ['메이25', '그냥 그랬어요,'], ['메이25', '감바스는 쏘쏘예요 에그베네딕트는 맛있어요'], ['메이25', '굿'], ['메이25', '언제나 먹어도 맛있는 메이25~☆ 영일대해수욕장에 아이랑 갈만한 맛난 브런치카페예요~ 해물빠에야 한 입 스페인 여행~ 고기 가득 진한 굴라쉬, 따끈...\n내용 더보기'], ['메이25', '맛도 좋고 친절 + 깨끗! 오래 오래 뵈어요 :)'], ['메이25', '맛있어요 두툼한 베이컨도 너무 굿굿 👍서비스 초콜렛도 맛있고 꼭 빵추가 하세요!!!'], ['메이25', '굴라쉬 빵추가가 없어서 쪼금 아쉬웠어요ㅠㅠ'], ['메이25', '음식에 맛과 정성이 느껴집니다. 재료 또한 신선하고 좋은것만 사용하는 듯~'], ['메이25', '전에 커피만 마셔보고 커피맛집이라 오랜만에 다시 방문했는데 이번에는 브런치도 먹어봤어요~ 음식맛 도 좋네요^^ 커피는 더 맛있구요~~ 잘 먹었습니다~!\n내용 더보기'], ['메이25', '커피도 맛있고 브런치도 굿!!!'], ['메이25', '정성가득'], ['메이25', '음식 너무 맛나요'], ['메이25', '에그베네딕트 진짜 맛있어요~ 근데 빠에야는 짜고 케챱 맛이 너무 많이 나오'], ['메이25', '맛있고, 사장님이 친절해요.'], ['메이25', '브런치 맛있어요 넘나리 맛있게먹음 음식 향기 향기 롭궁 커피는 거품치는 연습이 필요하지만 그래두가 성비좋아용\n내용 더보기'], ['메이25', '좋아요'], ['메이25', '음식이 다 맛있어요~ 매장

In [6]:
print(len(search_blog_url_result))
print(search_blog_url_result)

1022
[['메이25', 'https://blog.naver.com/realhansai/222600525706'], ['메이25', 'https://blog.naver.com/deeplotus/222578355055'], ['메이25', 'https://blog.naver.com/lee_minjee91/222615939373'], ['메이25', 'https://blog.naver.com/eunhui0407/222607319633'], ['메이25', 'https://blog.naver.com/jkj880308/222597780395'], ['메이25', 'https://blog.naver.com/gjskdud1002/222603487025'], ['메이25', 'https://blog.naver.com/syp15/222600991220'], ['메이25', 'https://blog.naver.com/jejuna17/221988561791'], ['메이25', 'https://blog.naver.com/rqw59590/222592455401'], ['메이25', 'https://blog.naver.com/yj9340/221938505677'], ['메이25', 'https://blog.naver.com/hey8688/222586840808'], ['메이25', 'https://blog.naver.com/whity0721/222014322818'], ['메이25', 'https://blog.naver.com/dpsl0361/222587455655'], ['메이25', 'https://blog.naver.com/rose_0221/222028230198'], ['메이25', 'https://blog.naver.com/junglesj/222411193594'], ['메이25', 'https://blog.naver.com/zeonhs/222307072554'], ['메이25', 'https://blog.naver.com/dsa9195/222499027091'], ['

DB에 저장하기 전 리뷰 데이터에서 이모지들을 제거합니다.

In [9]:
for i in range(len(search_review_result)):
    search_review_result[i][1] = search_review_result[i][1].encode('cp949', 'ignore').decode('cp949')

# 데이터 저장

sqlite3를 이용해 크롤링한 데이터들을 저장합니다.   
- simple_info_table : 장소에 대한 간단한 정보들
- review_info_table :리뷰에 대한 정보들
- blog_url_table : 블로그 링크에 대한 정보들

In [10]:
import pandas as pd
import sqlite3

simple_result = pd.DataFrame(search_simple_result)
simple_result.columns = ['이름', '전화번호', '주소', '평점', '방문자수', '블로그리뷰수']

review_result = pd.DataFrame(search_review_result)
review_result.columns = ['이름', '리뷰']

blog_url_result = pd.DataFrame(search_blog_url_result)
blog_url_result.columns = ['이름', 'URL']


conn = sqlite3.connect('./navermap_search_result.db')
simple_result.to_sql('simple_info_table', conn, chunksize=1000)
review_result.to_sql('review_info_table', conn, chunksize=1000)
blog_url_result.to_sql('blog_url_table', conn, chunksize=1000)
conn.commit()
conn.close()

저장이 잘 되었는지 쿼리문을 통해 확인합니다.

In [12]:
conn = sqlite3.connect('navermap_search_result.db')
simple = pd.read_sql('select * from simple_info_table', conn)
review = pd.read_sql('select * from review_info_table', conn)
blog_url = pd.read_sql('select * from blog_url_table', conn)
conn.commit()
conn.close()

In [13]:
simple

,index,이름,전화번호,주소,평점,방문자수,블로그리뷰수
0,0,메이25,054-251-0525,경북 포항시 북구 두호로6번길 17 1층 103호,4.54,49,51
1,1,소디스커피,0507-1346-9667,경북 포항시 북구 삼호로 246,4.33,216,230
2,2,서정빌라,none,경북 포항시 북구 삼호로265번길 26,4.64,97,160
3,3,애니시,0507-1438-5591,경북 포항시 북구 학전로 142 1층 애니시,4.5,18,39
4,4,안녕겨울 영일대점,0507-1370-9559,경북 포항시 북구 삼호로 265 1층,4.4,74,91
5,5,투썸플레이스 포항북부점,054-255-2631,경북 포항시 북구 해안로 93,4.43,318,35
6,6,투썸플레이스 포항두호점,0507-1411-2122,경북 포항시 북구 두호로 26 투썸플레이스 포항두호점,4.49,404,9
7,7,달짝지근,010-4668-4662,경북 포항시 북구 삼호로265번길 23 달짝지근,4.43,43,22
8,8,배스킨라빈스 포항두호,054-242-3101,경북 포항시 북구 두호로 25 아이코아,4.5,854,10
9,9,빽다방 포항영일대점,010-7271-7696,경북 포항시 북구 삼호로 237,4.6,73,9


In [14]:
review

,index,이름,리뷰
0,0,메이25,10명 이상 참여하면\n방문 키워드 통계가 공개됩니다!
1,1,메이25,다 맛있어요.그중 난 빠에야
2,2,메이25,너무 맛있고 기분이 좋아져요. 대접받는 느낌이 막 풍 기는 좋은 곳이었어요. 저는 ...
3,3,메이25,좋아요
4,4,메이25,"음식은 깔끔. 직접 만든 치즈, 베이컨 등등"
...,...,...,...
1785,1785,더리터 포항두호점,가성비 좋습니다.
1786,1786,더리터 포항두호점,포항 더리터커피 더리터사이즈는 최고죠!
1787,1787,더리터 포항두호점,싸고 맛있어요
1788,1788,더리터 포항두호점,양도 많고 맛있어요


In [15]:
blog_url

,index,이름,URL
0,0,메이25,https://blog.naver.com/realhansai/222600525706
1,1,메이25,https://blog.naver.com/deeplotus/222578355055
2,2,메이25,https://blog.naver.com/lee_minjee91/222615939373
3,3,메이25,https://blog.naver.com/eunhui0407/222607319633
4,4,메이25,https://blog.naver.com/jkj880308/222597780395
...,...,...,...
1017,1017,더리터 포항두호점,https://blog.naver.com/musthave5/221765413892
1018,1018,더리터 포항두호점,https://blog.naver.com/dsa9195/221273096599
1019,1019,더리터 포항두호점,https://blog.naver.com/wjpling/221295262950
1020,1020,더리터 포항두호점,https://blog.naver.com/square36/221264612951


# 블로그 리뷰 크롤링

앞서 크롤링한 블로그 링크 정보들을 이용해 블로그 리뷰 데이터들을 가져옵니다.

In [39]:
import pandas as pd
import sqlite3

# DB에서 블로그 링크 정보를 가져옵니다.
conn = sqlite3.connect('./navermap_search_result.db')
blog_url = pd.read_sql('select * from blog_url_table', conn)
conn.commit()
conn.close()

blog_url

,index,이름,URL
0,0,메이25,https://blog.naver.com/realhansai/222600525706
1,1,메이25,https://blog.naver.com/deeplotus/222578355055
2,2,메이25,https://blog.naver.com/lee_minjee91/222615939373
3,3,메이25,https://blog.naver.com/eunhui0407/222607319633
4,4,메이25,https://blog.naver.com/jkj880308/222597780395
...,...,...,...
1017,1017,더리터 포항두호점,https://blog.naver.com/musthave5/221765413892
1018,1018,더리터 포항두호점,https://blog.naver.com/dsa9195/221273096599
1019,1019,더리터 포항두호점,https://blog.naver.com/wjpling/221295262950
1020,1020,더리터 포항두호점,https://blog.naver.com/square36/221264612951


In [41]:
print(len(blog_url))

for i in range(len(blog_url)):
    if 'blog' in blog_url.loc[i][2]:
        a = 1
    elif 'cafe' in blog_url.loc[i][2]:
        print(blog_url.loc[i][2])
    else:
        print(blog_url.loc[i][2])

1022
http://cafe.naver.com/kks100/2437
http://cafe.naver.com/wedding052/166662
http://post.naver.com/viewer/postView.nhn?volumeNo=16040180&memberNo=878677&vType=VERTICAL
http://cafe.naver.com/hotellife/1588352
http://cafe.naver.com/doralife/2370
http://cafe.naver.com/cafedoux/35
http://post.naver.com/viewer/postView.nhn?volumeNo=17158384&memberNo=22378852&vType=VERTICAL
https://post.naver.com/viewer/postView.nhn?volumeNo=29781463&memberNo=41592176&vType=VERTICAL
http://cafe.naver.com/cafeflex/8378
http://cafe.naver.com/hmckorea/441248


blog가 아닌 url은 iframe 및 태그 정보가 달라 탐색에서 제외합니다.   
개수 또한 기존 1022개의 데이터 중 10개 정도 해당하기 때문에 제외하여도 결과에 미치는 영향은 미미할 것입니다.

In [68]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import datetime
import requests

driver_path = './driver/chromedriver'
driver = webdriver.Chrome(executable_path=driver_path)

blog_text = []
for i in range(len(blog_url)):
    name = blog_url.loc[i][1]
    url = blog_url.loc[i][2]

    page_type = 'none'
    if not 'blog' in url:
        continue
    
    driver.get(url)
    time.sleep(3)

    # frame 전환
    search_frame = driver.find_element_by_id('mainFrame')
    driver.switch_to.frame(search_frame)
    
    btxt = driver.find_element_by_id('post_1').text

    blog_text.append([name, btxt])

C:\Users\namke\AppData\Local\Temp/ipykernel_10516/716679470.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)
C:\Users\namke\AppData\Local\Temp/ipykernel_10516/716679470.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_frame = driver.find_element_by_id('mainFrame')
C:\Users\namke\AppData\Local\Temp/ipykernel_10516/716679470.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  blog_text = driver.find_element_by_id('post_1').text


장소의 이름과 해당 장소에 대한 모든 blog의 text들을 가져왔습니다.   
이를 DB에 저장하기 전 불필요한 앞부분과 뒷부분들을 제거합니다.

In [ ]:
blog_text

먼저 text를 보면 이웃추가가 반복되는 것으로 보입니다.   
이를 기준으로 모든 text에서 동일한 패턴이 있는지 확인합니다.

In [14]:
no_with = []
for per in blog_text:
    if not '이웃추가' in per:
        no_with.append(per)
len(no_with)

64

In [15]:
cnt = 0
for per in no_with:
    if not '지도로 보기' in per:
        cnt = cnt + 1
cnt

0

탐색 결과 blog에서 가져온 text들은 '이웃추가'가 상단에 없는 text도 있어, 대신 '지도로 보기'로 탐색하니 모든 경우를 고려하게 됩니다.   
'이웃추가'와 '지도로 보기'를 이용해 blog 데이터에서 필요없는 앞 부분을 제거합니다.

In [20]:
tt[0].find('지도로 보기')
# # 지도로 보기는 6글자이므로 위 결과에 +7이 시작 부분
# 95이면 102부터

tt[1].find('이웃추가')
# 이웃추가는 4글자이므로 +4가 시작 부분
# 55이면 59부터

55

불필요한 뒷부분은 주로 검색을 위한 태그로 시작되는 것으로 보여 이를 기준으로 제거합니다.

In [36]:
import re

# 정규표현식으로 마지막 태그 이후 부분을 모두 제거
m = re.findall(r'[#]\w+', tt[10])
ff = tt[10].find(m[0])
tt[10][ff:ff+1000]

'#포항두호동맛집 #포항브런치 #포항브런치맛집 #메이25 #may25 #포항메이25 #포항북구카페 #포항카페추천 #포항카페 #포항두호동카페 #두호동맛집 #포항두호동맛집 #포항브런치 #포항브런치맛집 #메이25 #may25 #포항메이25 #포항북구카페 #포항카페추천 #포항카페 #포항두호동카페 #두호동맛집 ` 공감 16 댓글 10 Keep 메모 보내기            인쇄'

마무리로 text에서 불필요한 문자들(줄넘김 등)을 제거합니다.

전처리한 blog 리뷰 데이터들을 DB에 저장합니다.

In [ ]:
conn = sqlite3.connect('./navermap_search_result.db')
blog_text.to_sql('blog_info_table', conn, chunksize=1000)
conn.commit()
conn.close()

DB에 저장이 잘 되었는지 확인합니다.

In [ ]:
conn = sqlite3.connect('./navermap_search_result.db')
blog = pd.read_sql('select * from blog_info_table', conn)
conn.commit()
conn.close()